<a href="https://colab.research.google.com/github/yahyahpa/Check-Digit-Program/blob/master/Encrypted_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Encrypted Deep Learning
---
Traing a Deep Learning Model
---
Let's build and train a toy deep learning model

In [0]:
import torch as th
from torch import nn
from torch import optim
import torch.nn.functional as F


In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[-1.6],[1.]], requires_grad=True)


In [3]:
encrypted_target = target.fix_precision()
encrypted_target

AttributeError: ignored

In [4]:
encrypted_target.float_precision()

NameError: ignored

In [0]:
# a toy model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).mean()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        

In [0]:
model = Net()


In [7]:
train()

tensor(0.9193)
tensor(0.8384)
tensor(0.7895)
tensor(0.7452)
tensor(0.7050)
tensor(0.6650)
tensor(0.6259)
tensor(0.5885)
tensor(0.5518)
tensor(0.5154)
tensor(0.4807)
tensor(0.4515)
tensor(0.4227)
tensor(0.3947)
tensor(0.3685)
tensor(0.3485)
tensor(0.3280)
tensor(0.3086)
tensor(0.2903)
tensor(0.2742)


In [8]:
predictions = model(data)
predictions

tensor([[-0.3635],
        [ 0.3417],
        [-0.8945],
        [ 0.4554]], grad_fn=<AddmmBackward>)

Encrypted Deep Learning using PySyft
---
Now let's use the library PySyft, which we introduced in the first section of this course, to cary out encrypted learning :


In [9]:
!pip install syft

     |████████████████████████████████| 337kB 6.9MB/s 
     |████████████████████████████████| 10.1MB 25.3MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 773.1MB 24kB/s 
     |████████████████████████████████| 256kB 38.5MB/s 
     |████████████████████████████████| 204kB 61.2MB/s 
     |████████████████████████████████| 471kB 38.9MB/s 
     |████████████████████████████████| 399kB 52.3MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 122kB 45.5MB/s 
  Created wheel for zstd: filename=zstd-1.4.4.0-cp36-cp36m-linux_x86_64.whl size=1131186 sha256=43a0ac0defc8aaf0f91de8d1131a359ea5c15d6ba959ca85eb92e7df264b8698
  Stored in directory: /root/.cache/pip/wheels/ce/0d/5a/3c2ccf5522d1cd3ecdeb2af8c9889179d7a204376b1d5c042d
Successfully built zstd
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
  Found existing installation: torch

In [10]:
import syft as sy
hook = sy.TorchHook(th)

In [11]:
# create a couple of workers
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

# this step is important in the real-world application.
# You need to inform the workers of others existance
# you will probably have an ssh or http worker, not a virtual worker.

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker me already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [15]:
# encrypt the model and share it among participants
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

AttributeError: ignored

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [16]:
encrypted_prediction = encrypted_model(encrypted_data)
encrypted_prediction

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:83529279717 -> alice:73643041711]
	-> [PointerTensor | me:42139459576 -> bob:22347933232]
	*crypto provider: secure_worker*

In [17]:

results = encrypted_prediction.get()
results

(Wrapper)>FixedPrecisionTensor>tensor([[-364],
        [ 340],
        [-893],
        [ 453]])

In [18]:

results = results.float_precision()
results

tensor([[-0.3640],
        [ 0.3400],
        [-0.8930],
        [ 0.4530]])

Additive Secrete Sharing
---


Additive Secret Sharing is a protocol for Multi-Party Computation. It allows multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x.
Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [19]:

x = 5
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted).

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [20]:

bob_x_share *= 2 
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. 
Decryption happens by summing all the shares together MODULUS some constant:

In [21]:

x = 5

Q = 23740629843760239486723 # large prime number

bob_x_share = 23552870267 # <- a random number


alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [22]:

(bob_x_share + alice_x_share) % Q

5

 Fixed Precision Encoding
 ---
 Additive secrete sharing works with integers. Thus, to apply this protocol on deep learning, we first need to convert the gradeints to integers. To do so, we used a predifned funciton fixed_precision. In the following, we illustrate its internat implemention in a very simple appraoch:

In [0]:
BASE=10
PRECISION=4

def encode(x):
    return int((x * (BASE ** PRECISION)) % Q)

def decode(x):
    return (x if x <= Q/2 else x - Q)  / BASE**PRECISION
    

In [24]:
encode(-10)

23740629843760239386723

In [25]:
decode(23740629843760239386723)

-10.0

In [26]:
a = a.torch

NameError: ignored

---
Apply Addivite Secrete Sharing using PySyft on a real dataset (MNIST or CIFAR10)
---

In [31]:
from torchvision import datasets, transforms
model = "/content/drive/My Drive/Colab Notebooks/MyTrainedModel1.pth"


dataset = th.utils.data.DataLoader(
    datasets.FashionMNIST('../data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            ])), 
        batch_size=1, shuffle=True)

encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)
encrypted_data = dataset.fix_precision().share(alice, bob, crypto_provider=secure_worker)

encrypted_prediction = encrypted_model(encrypted_data)
encrypted_prediction
results = encrypted_prediction.get()
results
results = results.float_precision()
results
x = 5
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x
bob_x_share *= 2 
alice_x_share *= 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x
x = 5

Q = 23740629843760239486723 # large prime number

bob_x_share = 23552870267 # <- a random number


alice_x_share = Q - bob_x_share + x
alice_x_share
(bob_x_share + alice_x_share) % Q

  0%|          | 49152/26421880 [00:00<00:54, 480853.93it/s]

26427392it [00:00, 81067760.18it/s]                            


Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw


32768it [00:00, 590178.31it/s]
 20%|█▉        | 868352/4422102 [00:00<00:00, 8116660.54it/s]

Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw


4423680it [00:00, 30843212.34it/s]                           
8192it [00:00, 192689.08it/s]


Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw
Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw
Processing...
Done!


AttributeError: ignored